# Get MIRI programs info #

**Goal:** Create a csv file with relevant information on all observed MIRI programs

**Author:** Boris Trahin, Staff Scientist II, MIRI team

**Last updated:** November 4, 2024

***
### Import useful packages

In [6]:
from astroquery.mast.missions import MastMissions
from astroquery.mast import Observations
from astropy.table import unique, vstack, Table
from datetime import date
import pandas as pd
update = date.today()

***
### Get entire catalog of MIRI observations

<div class="alert alert-block alert-info">
Set some parameters
</div>

In [7]:
save = True # save the catalogs into csv file

<div class="alert alert-block alert-info">
Get the data from MAST. This is done for each exposure types (otherwise simultaneous observations are badly referenced)
</div>

In [8]:

good_columns = ['fileSetName', 'obs_id', 'program', 'proposal_cycle', 'date_obs', 'date_beg', 'date_end',
                'visitgrp', 'observtn', 'bkgdtarg', 'visit',
                'targtype', 'exp_type', 'tsovisit', 'expripar', 'detector',
                'opticalElements', 'filter', 'subarray', 'pps_aper', 'apername',
                'miri_channel', 'miri_band', 'miri_coronmsk', 'targ_ra', 'targ_dec',
                'nresets', 'frmdivsr', 'nframes', 'ngroups', 'nints', 'readpat', 'numdthpt',
                'patttype', 'miri_dithopfr', 'miri_numdsets',
                'tframe', 'tgroup', 'effinttm','duration', 'eng_qual',
                'hga_move', 'lamp', 'pcs_mode', 'miri_cccstate', 'exp_only', 'visitsta', 'template',
                'targdesc', 'scicat', 'targcat', 'targprop', 'targname', 'obslabel', 'title',
                'pi_name', 'category', 'access']

exposure_types = ['MIR_DARKMRS', 'MIR_DARKIMG',
                  'MIR_FLATIMAGE', 'MIR_FLATMRS', 'MIR_FLATIMAGE-EXT',
                  'MIR_MRS', 'MIR_IMAGE', 
                  'MIR_LRS-SLITLESS', 'MIR_LRS-FIXEDSLIT',
                  'MIR_4QPM','MIR_LYOT','MIR_CORONCAL', 
                  'MIR_TACQ', 'MIR_TACONFIRM']

missions = MastMissions(mission='jwst')
dfs = []

for e in exposure_types:
    print(f"Getting {e} exposures...")
    results = missions.query_criteria(select_cols=good_columns,
    exp_type = e,
    instrume='MIRI',
    limit=-1,
        )
    sub_df = results.to_pandas()
    dfs.append(sub_df)
df = pd.concat(dfs)


Getting MIR_DARKMRS exposures...


Getting MIR_DARKIMG exposures...
Getting MIR_FLATIMAGE exposures...
Getting MIR_FLATMRS exposures...
Getting MIR_FLATIMAGE-EXT exposures...
Getting MIR_MRS exposures...
Getting MIR_IMAGE exposures...
Getting MIR_LRS-SLITLESS exposures...
Getting MIR_LRS-FIXEDSLIT exposures...
Getting MIR_4QPM exposures...
Getting MIR_LYOT exposures...
Getting MIR_CORONCAL exposures...
Getting MIR_TACQ exposures...
Getting MIR_TACONFIRM exposures...


<div class="alert alert-block alert-info">
Uncomment the following line to save the list of programs before the changes made in the next cell
</div>

In [ ]:
# df.to_csv('./MIRI_programs_MAST.csv', index=False) # save to a csv file and remove first index column

<div class="alert alert-block alert-info">
Some columns are badly reference (e.g. MRS detector), so organize it.
One can uncomment the line df2 = df2.drop_duplicates(subset=['obs_id', 'exp_type']) to remove duplicates based on obs_id and exp_type. This will keep only 1 dither position per exposure.
</div>

In [10]:
df2 = df.reindex(good_columns, axis=1)

# Some columns are badly referenced
df2.loc[(df2['exp_type']=='MIR_MRS'), 'detector'] = 'MIRIFUSHORT, MIRIFULONG'
df2.loc[(df2['exp_type']=='MIR_DARKMRS'), 'detector'] = 'MIRIFUSHORT, MIRIFULONG'

# df2['detector'] = df2['detector'].fillna('')
# df2['detector'] = df2['detector'].astype(str)
# df2['miri_channel'] = df2['miri_channel'].fillna('')
# df2['miri_channel'] = df2['miri_channel'].astype(str)

df2.loc[(df2['detector']=='MIRIFUSHORT, MIRIFULONG'), 'miri_channel'] = '12, 34'
# df2.loc[(df2['opticalElements'].str.contains('CH12')) & (df2['opticalElements'].str.contains('CH34')) & (df2['detector'].str.contains('MIRIFU')), 'miri_channel'] = '12, 34'

df2['readpat'] = df2['readpat'].fillna('')
df2['readpat'] = df2['readpat'].astype(str)
df2.loc[(df2['tframe']<3) & (df2['frmdivsr']==1) & (df2['nresets']==1), 'readpat'] = 'FASTR1'
df2.loc[(df2['tframe']<3) & (df2['frmdivsr']==1) & (df2['nresets']==0), 'readpat'] = 'FAST'
df2.loc[(df2['tframe']<3) & (df2['frmdivsr']==4), 'readpat'] = 'FASTGRPAVG'
df2.loc[(df2['tframe']<3) & (df2['frmdivsr']==8), 'readpat'] = 'FASTGRPAVG8'
df2.loc[(df2['tframe']<3) & (df2['frmdivsr']==32), 'readpat'] = 'FASTGRPAVG32'
df2.loc[(df2['tframe']<3) & (df2['frmdivsr']==64), 'readpat'] = 'FASTGRPAVG64'

df2.loc[(df2['tframe']>3) & (df2['frmdivsr']==1) & (df2['nresets']==1), 'readpat'] = 'SLOWR1'
df2.loc[(df2['tframe']>3) & (df2['frmdivsr']==1) & (df2['nresets']==0), 'readpat'] = 'SLOW'

df2 = df2.reset_index()
df2 = df2.drop(df2[df2['fileSetName'].str.contains('-')].index)

df2 = df2.sort_values(['program', 'observtn'],axis=0, ascending=False) # latest program first
# df2 = df2.drop_duplicates(subset=['obs_id', 'exp_type']) # Uncomment this line to remove duplicates based on obs_id and exp_type. This will keep only 1 dither position per exposure.
df2 = df2.reindex(good_columns, axis=1)
if save:
    with open(f"./JWST_MIRI_programs_info_updated{date.today()}.csv", 'w') as outfile:
        df2.to_csv(outfile, index=False) # save to a csv file and remove first index column